In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
import gzip
import shutil
from struct import unpack
from collections import namedtuple, Counter, defaultdict
from pathlib import Path
from urllib.request import urlretrieve
from urllib.parse import urljoin
from datetime import timedelta
from time import time

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

In [14]:
def format_time(t):
    ''' 
    Return a formatted time string (HH:MM::SS) based on a numeric time() value
    '''
    m, s = divmod(t,60)
    h, m = divmod(m, 60)

    return f'{h:0>2.0f}:{m:0>2.0f}:{s:0>5.2f}'

In [23]:
data_path = Path('data')
itch_store = str(data_path / 'itch.h5')
order_book_store = data_path / 'order_book.h5'      

data\order_book.h5


In [24]:
HTTPS_URL = 'https://emi.nasdaq.com/ITCH/Nasdaq%20ITCH/'
SOURCE_FILE = '10302019.NASDAQ_ITCH50.gz'

In [27]:
def may_be_download(url):
    ''' 
    Download & unzip ITCH data if not yet available
    '''
    if not data_path.exists():
        print("Creating Directory")
        data_path.mkdir()
    else:
        print("Directory Exists")

    filename = data_path / url.split('/')[-1]
    if not filename.exists():
        print("Downloading...", url)
        urlretrieve(url, filename)
    else:
        print("File Exists")

    unzipped = data_path / (filename.stem + '.bin')
    if not unzipped.exists():
        print('Unzipping to', unzipped)
        with gzip.open(str(filename), 'rb') as f_in:
            with open(unzipped, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    else:
        print("File Already Unpacked")
    return unzipped

In [ ]:
file_name = may_be_download(urljoin(HTTPS_URL, SOURCE_FILE))
date = file_name.name.split('.')[0]